In [23]:
import base64
import random
import tensorflow as tf
print(tf.__version__)

2.0.0


In [24]:
## Xs'
M = 500
N = 6 # 6 Bytes
def data_gen(m = M , n = N) :
    X_int   = [ random.getrandbits(n*8) for _ in range(m) ]
    X_bits  = [ [ 1 if num & (1 << pow2) else 0 for pow2 in range(n*8) ] for num in X_int ]
    X_bytes = [ num.to_bytes(n, 'little') for num in X_int ] 
    return X_bits, X_bytes
random.seed(0)
X, X_bytes = data_gen()
print(X[:2])
print(X_bytes[:2])


[[1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]]
[b'\xcd\x07,\xd8\x9fb', b'\xacL\t\xc2\xe7\xe3']


In [25]:
## Ys'
Y_bytes = [ base64.b64encode(s) for s in X_bytes ]
Y_int   = [ int.from_bytes(s, 'little')   for s in Y_bytes ]
Y       = [ [1 if num & 1 << pow2 else 0 for pow2 in range(8*8) ] for num in Y_int ]


print(Y_bytes[:2])
print(Y[:2])
print(base64.b64decode(Y_bytes[0]))

[b'zQcs2J9i', b'rEwJwufj']
[[0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]]
b'\xcd\x07,\xd8\x9fb'


In [18]:
M_train, M_test  = int(M*0.8), int(M*0.2)
assert M_train + M_test == M
# we are trying to reverse it
X_train, X_test  = tf.convert_to_tensor(Y[:M_train]), tf.convert_to_tensor(Y[M_train:])
Y_train, Y_test  = tf.convert_to_tensor(X[:M_train]), tf.convert_to_tensor(X[M_train:])

print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)

X_train:  (400, 64)
Y_train:  (400, 48)


In [ ]:
ACTIVATION = 'relu'
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=[64,]),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(48, activation=ACTIVATION)
])
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.compile(optimizer='adam', loss='mse')
model.summary()
model.fit(X_train, Y_train, epochs=20000)
predictions = [ [ 1 if bit_predict >= 0.5 else 0 for bit_predict in predict ] for predict in model.predict(X_test) ]
print("Done " * 100)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_26 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_27 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_28 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_29 (Dense)             (None, 48)                6192      
Total params: 51,696
Trainable params: 51,696
Non-trainable params: 0
_________________________________________________________________
Train on 400 samples
Epoch 1/20000
400/400 [==============================] - 0s 717us/sample - loss: 0.3507
Epoch 2

400/400 [==============================] - 0s 33us/sample - loss: 0.0140
Epoch 83/20000
400/400 [==============================] - 0s 28us/sample - loss: 0.0134
Epoch 84/20000
400/400 [==============================] - 0s 32us/sample - loss: 0.0129
Epoch 85/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0126
Epoch 86/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0122
Epoch 87/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0117
Epoch 88/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0112
Epoch 89/20000
400/400 [==============================] - 0s 31us/sample - loss: 0.0110
Epoch 90/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0106
Epoch 91/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0102
Epoch 92/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0098
Epoch 93/20000
400/400 [=======================

400/400 [==============================] - 0s 32us/sample - loss: 0.0028
Epoch 175/20000
400/400 [==============================] - 0s 29us/sample - loss: 0.0028
Epoch 176/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0028
Epoch 177/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0029
Epoch 178/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0029
Epoch 179/20000
400/400 [==============================] - 0s 35us/sample - loss: 0.0028
Epoch 180/20000
400/400 [==============================] - 0s 28us/sample - loss: 0.0030
Epoch 181/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0033
Epoch 182/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0035
Epoch 183/20000
400/400 [==============================] - 0s 34us/sample - loss: 0.0036
Epoch 184/20000
400/400 [==============================] - 0s 33us/sample - loss: 0.0043
Epoch 185/20000
400/400 [============

400/400 [==============================] - 0s 28us/sample - loss: 0.0029
Epoch 267/20000
400/400 [==============================] - 0s 31us/sample - loss: 0.0028
Epoch 268/20000
400/400 [==============================] - 0s 31us/sample - loss: 0.0028
Epoch 269/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0026
Epoch 270/20000
400/400 [==============================] - 0s 29us/sample - loss: 0.0026
Epoch 271/20000
400/400 [==============================] - 0s 32us/sample - loss: 0.0025
Epoch 272/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0024
Epoch 273/20000
400/400 [==============================] - 0s 32us/sample - loss: 0.0025
Epoch 274/20000
400/400 [==============================] - 0s 29us/sample - loss: 0.0024
Epoch 275/20000
400/400 [==============================] - 0s 28us/sample - loss: 0.0024
Epoch 276/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0025
Epoch 277/20000
400/400 [============

400/400 [==============================] - 0s 34us/sample - loss: 0.0035
Epoch 359/20000
400/400 [==============================] - 0s 32us/sample - loss: 0.0036
Epoch 360/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0035
Epoch 361/20000
400/400 [==============================] - 0s 28us/sample - loss: 0.0037
Epoch 362/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0035
Epoch 363/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0033
Epoch 364/20000
400/400 [==============================] - 0s 30us/sample - loss: 0.0033
Epoch 365/20000
400/400 [==============================] - 0s 31us/sample - loss: 0.0033
Epoch 366/20000
400/400 [==============================] - 0s 31us/sample - loss: 0.0033
Epoch 367/20000
400/400 [==============================] - 0s 37us/sample - loss: 0.0032
Epoch 368/20000
400/400 [==============================] - 0s 37us/sample - loss: 0.0029
Epoch 369/20000
400/400 [============

...

400/400 [==============================] - 0s 29us/sample - loss: 3.1856e-04
Epoch 17054/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.1785e-04
Epoch 17055/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.1770e-04
Epoch 17056/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.1747e-04
Epoch 17057/20000
400/400 [==============================] - 0s 31us/sample - loss: 3.1665e-04
Epoch 17058/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.1677e-04
Epoch 17059/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.1666e-04
Epoch 17060/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.1651e-04
Epoch 17061/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.1630e-04
Epoch 17062/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.1603e-04
Epoch 17063/20000
400/400 [==============================] - 0s 29us

400/400 [==============================] - 0s 29us/sample - loss: 3.3670e-04
Epoch 17141/20000
400/400 [==============================] - 0s 31us/sample - loss: 3.3738e-04
Epoch 17142/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.3687e-04
Epoch 17143/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.3408e-04
Epoch 17144/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.3186e-04
Epoch 17145/20000
400/400 [==============================] - 0s 28us/sample - loss: 3.2989e-04
Epoch 17146/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.2931e-04
Epoch 17147/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.3040e-04
Epoch 17148/20000
400/400 [==============================] - 0s 28us/sample - loss: 3.3073e-04
Epoch 17149/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.3153e-04
Epoch 17150/20000
400/400 [==============================] - 0s 30us

400/400 [==============================] - 0s 33us/sample - loss: 3.6047e-04
Epoch 17228/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.6268e-04
Epoch 17229/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.6863e-04
Epoch 17230/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.7019e-04
Epoch 17231/20000
400/400 [==============================] - 0s 28us/sample - loss: 3.6698e-04
Epoch 17232/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.6863e-04
Epoch 17233/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.6594e-04
Epoch 17234/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.6660e-04
Epoch 17235/20000
400/400 [==============================] - 0s 28us/sample - loss: 3.6374e-04
Epoch 17236/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.6480e-04
Epoch 17237/20000
400/400 [==============================] - 0s 34us

400/400 [==============================] - 0s 32us/sample - loss: 3.5707e-04
Epoch 17315/20000
400/400 [==============================] - 0s 31us/sample - loss: 3.5785e-04
Epoch 17316/20000
400/400 [==============================] - 0s 31us/sample - loss: 3.6039e-04
Epoch 17317/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.6296e-04
Epoch 17318/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.6701e-04
Epoch 17319/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.7295e-04
Epoch 17320/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.7975e-04
Epoch 17321/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.8901e-04
Epoch 17322/20000
400/400 [==============================] - 0s 29us/sample - loss: 4.0175e-04
Epoch 17323/20000
400/400 [==============================] - 0s 29us/sample - loss: 4.0758e-04
Epoch 17324/20000
400/400 [==============================] - 0s 29us

400/400 [==============================] - 0s 29us/sample - loss: 3.3696e-04
Epoch 17402/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.4121e-04
Epoch 17403/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.4878e-04
Epoch 17404/20000
400/400 [==============================] - 0s 29us/sample - loss: 3.6539e-04
Epoch 17405/20000
400/400 [==============================] - 0s 28us/sample - loss: 3.7621e-04
Epoch 17406/20000
400/400 [==============================] - 0s 30us/sample - loss: 3.9078e-04
Epoch 17407/20000
400/400 [==============================] - 0s 31us/sample - loss: 4.1295e-04
Epoch 17408/20000
400/400 [==============================] - 0s 34us/sample - loss: 4.3480e-04
Epoch 17409/20000
400/400 [==============================] - 0s 28us/sample - loss: 4.6578e-04
Epoch 17410/20000
400/400 [==============================] - 0s 29us/sample - loss: 5.0728e-04
Epoch 17411/20000
400/400 [==============================] - 0s 30us

In [14]:
print(predictions[0])
print(Y_test[0])
# correct = sum([ int(''.join([str(b) for b in x]),2) == int(''.join([str(b) for b in y]), 2) for x,y in zip(predictions, Y_test)])
correct  = sum([ y_hat == y for y_hat, y in zip(predictions, Y_test)])
accuracy = correct / float(M_test)
print("Correct cases : {}/{}".format(correct, M_test) )
print("accuracy      : {}%".format(accuracy* 100.0 ) )

[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
Correct cases : 0/100
accuracy      : 0.0%
